Import our dependencies and the Minitorch from my github

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import sys
import os
from tqdm import tqdm
import pickle


from engine.minitorch import Tensor,optim

I will put a seed right now for the reproducibility of the project.

In [3]:
np.random.seed(3329)

This are the variables that we need in order to create the data folder with the dataset inside,because I can't upload the repository with the dataset of Shakespeare pre-charged.

In [4]:
data_directory = "data"
file_path = os.path.join(data_directory, "input.txt")
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

This will create a data folder if it doesn't exists

In [5]:
if not os.path.exists(data_directory):
    
    os.makedirs(data_directory)

if the inside of the data folder that we just created is empty we throw a request to the url for the dataset of Shakespeare of the github of Karpathy,raise for status to inform us what happened with the request and then open it indicating to the computer that need to transform it binary to avoid missinterpretations of characters and then finally write it down to the "input.txt" inside our data folder

In [6]:
if not os.path.exists(file_path):
    
    response = requests.get(url)
    
    response.raise_for_status()


    with open(file_path, "wb") as f:

        f.write(response.content)

        

Here what we do is open the input.txt of Shakespeare preaviously stored in the data folder and read it,the encoding utf-8 is the standard to read special characters like emojis,commas,dots,etc.\
Then the text variable is just all that concatenated ,like a list of all Shakespeare.

In [7]:
with open(file_path, 'r', encoding='utf-8') as f:

    text = f.read()

First we convert the text into a set,this will give us every character inside the text.\
Then we transform that set into a list to be able to sorted in order that the same character never change its index.\
Lastly we can see the len of the vocabulary and print it.

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)


print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


To tokenize them we just create two dictionaries, String to Integer(stoi) and Integer to String(itos).

One will have as key_value the character and as the value the index of that character in the list of chars.

And the other one will have its key_value the index and as value the character of that index in the list of chars.

This is exactly what we want because we use the list of characters and its indexes to transform them into integers and vice versa.

In [9]:
stoi = {ch:i for i,ch in enumerate(chars)}

itos = {i:ch for i,ch in enumerate(chars)}

Now here is the actual encoding and decoding.\
We gonna use lambda function because this is simple and with lambda we avoid writing a def just for this.

encoder = here we pass to lamba a string(s) then we ask for its correspondent value inside the stoi dictionary.

decoder = this will receive a list of numbers,search what character correspond to that integer inside of itos and concatenate them into a string.

In [10]:
encoder = lambda s:[stoi[c] for c in s]

decoder = lambda l: ''.join([itos[i]for i in l])

Now that we got our encoder and decoder que can transform the text into a numpy array.

In [ ]:
all_data = np.array(encoder(text), dtype=np.int64)

Here we gonna take the 90% of the total data(len of total_data * 0.9).\
And then we use that number to divide the data into train_data and test_data.

train_data being the first 90% of the data.\
test_data being the last 10% of the data.

In [ ]:
n = int(0.9 * len(all_data))

train_data = all_data[:n]
test_data = all_data[n:]

This is momentary and its something that you can change to have a different speed or precision in training.\
I realize that I need this 2 variables because of the creation of the parameters for the feed-foward network later on in the creation of the foward pass functions,so I decide to put this right up here(In the training preparation you can change the values according to how you want to train the model).

In [ ]:
block_size = 8
batch_size = 4

Here is the function to create batches because for training you can't pass all the data at once,explanation = Self-explanatory, just don't.

For that reason we create this function passing as arguments the split which is a string.\
The batch_size which is one of the dimensions and the block_size which is the other dimension.

First we declare the variable data, this variable will be equal to the train_data if when we pass the split(the string parameter) we said "train" otherwise the variable data will be equal to the test_data.

Now we gonna create the randomness in the batches in order to train better the model,using numpy,we tell it to give us a random number between 0 and the total of the data minus the block size so it doesn't give us por example the last 2 tokens in the data and broke our block size.\
And then we also pass the batch_size to tell numpy that we want more than a single number.

Next we gonna create our Tensors2D(block_size,batch_size).\
To create our inputs (x) we need to put them in a stack in order to create the mesh/matrix so using and excel sheet as analogy you can interpret the block size as the how large will we the rows and the batch_size as how large will be the columns(for example, if we pass block_size=8 and batch_size=4 the tensor that we gonna get is a 4x8 matrix,4 batches with a lenght of 8 values each).\
Once acquire the inputs, the targets(y) follow the same creation logic but +1.It will be the same size as the tensor "X".
Example: if tensor start with something like  43,56,21 then tensor Y will be like 56,21,33 .

Lastly we just return both the inputs and targets tensors.

In [ ]:
def get_batches(split,batch_size,block_size):

    data = train_data if split == "train" else test_data

    idx = np.random.randint(0,len(data) - block_size,batch_size)

    x = np.stack(data[i:i+block_size] for i in idx)

    y = np.stack(data[i + 1 : i + block_size + 1] for i in idx)

    return x, y

Here we have our variables to be able to create the table of embedding.\
We will use this to transform the 65 posibles tokens into vectors of random values(one vector per char) to be able to do maths operations with the vectors.

First we take the vocab_size.\
Then we assing a variable called numbers of embedding which is arbitrary,for a model like this I recommend 128 because is a tiny model,every of the 128 will give to the model context of what is about the token, for example, if the token 5 represent the letter "a" the table of embedding with each of the 128 will give us an adjective of it: "vocal","minuscule".


Then we initialize the variable table creating a tensor2D of random numbers,each row will represent 1 token(65 row because of the vocab_size) and every column is value of embedding for that token(128 embeddings per token).Creating a tensor2D of (65,128).\
We just multiply those values by the Xavier formula that is commonly used for Softmax/Sigmoid/Tanh because the randomness is too high for the model to start learning right away,it could probably(and luckily if doesn't explode)take a lot of epochs start learning.

By the way,we created this Variables outside the get_embeddings function because we need to train this random values,if we initialize them inside the function they will be created every time we call the function and never train.


Every parameter should be a Tensor of minitorch to be able to train it with Adam.

In [ ]:
vocab_size = vocab_size # Just to bring it closer we already know that they are 65 characters

n_embd = 128


token_table = Tensor( np.random.randn(vocab_size,n_embd) * np.sqrt(2/n_embd), requires_grad = False )

This short function is the embedding of our tokens,is short because Numpy does the hard lifting for us creating the new tensor and all that.

When we call the function we pass our batches of tokens for example (4x8) and we also pass the table of embeddings of random values(65,128).\
What Numpy does is create a new Tensor, but this tensor is a Tensor3D.\
When we said use the inputs as index of table we practically say,from the embeddings table put in a new tensor the rows of every value in the block_size of the batch_size giving us a Tensor3D of (4,8,128).\
In an excel sheet example in this new tensor we got: the 128 embeddings columns(n_embd=128) for each row of the 8 rows(block_size=8) and once we got that tensor2D we need to stack 4 tensor2D like those(batch_size=4) on top of each other.

In [ ]:
#  def get_embeddings(xb,token_table):
#      return token_table[xb]

Same as the token_table we iniliazite a tensor2D but instead of using the vocab_size we use the block_size(Also multiplying by the Xavier formula).

Every parameter should be a Tensor of minitorch to be able to train it with Adam.

In [ ]:
pos_table = Tensor( np.random.randn(block_size,n_embd) * np.sqrt(2/n_embd), requires_grad = False )

I encourage you to learn the theory behind this positional encoding,is facinating see how words can follow a sinusoidal wave to know time and position of each embedding(maybe in future projects I'll implement this advance,more mathematical version on a transformer,which is the original way suggested by the paper "Atenttion is all you need",extrapolation, "infinite" positions),For now I'll stick with the GPT way(Learned positional embeddings)which is just another tensor like the "embeddings_table" that add time and position with random numbers that later on the model will learn.\
Because the model without the positional encoding will think that "The dog bites the man" is the same as "The man bites the dog".


Here we take as parameters the stack batch(4,8) the token table and pos_table that we previusly have created.

For future operations I'll take the shape of xb .\
Now to make things more modular just in case that in the future I want a block_size of 16 instead of one of 8 like right now,we create an numpy array with the length of our block_size(T=8).

Then we have the token_table(65,128) but we need a tensor3D of (4,8,128 like the token_embeddings of the previous function).\
so we use the same fancy indexing trick to give us the tensor3D.(after a conclusion I find the previous functon redundant and I comment it, so it doesn't affect when running all the cells,just in case;But I will leave it right there because I see it as a sign of learn not as a mistake).

Then this next part is just to "name" put "labels" to every row in the pos_table,for example the first row of pos_table will be named position:0(this a analogy of how the model will think) and the next row position:1, this will continue until position:7 because the block_size only have 8 positions.\
This will give to the model a perception of time/space because this value will be train to tell the posible position of the token after seeing all the Shakespeare train data.

Next and finally is just the summ of the token_embd which give us "characteristics" of the token with the pos_embd that will give us "time/space" of the token,if you are wondering how two tensor with different sizes can do an addition to each other,the answer is : Because of numpy,numpy is doing the broadcasting so we don't have problems with the dimension,he es just doing again and again the addition of the tensor2D(pos_embd) the amount of batches of the tensor3D(4),so we ended up with a (4,8,128) tensor.

In [ ]:
def positional_encoding(xb,token_table,pos_table):

    B,T = xb.shape

    pos_idx = np.arange(T)

    token_embd = token_table[xb] # .data because that is the numpy array inside the Tensor of my Minitorch

    pos_embd = pos_table[pos_idx]

    x = token_embd + pos_embd

    return x

I'm using my minitorch but the "new parts" of the Transformer(positional encoding,multi-head atenttion,etc) I'll do it manual(including backprop).\
So for that reason I'll need a Softmax function.\
Quick recap(I've been explaining it in every project):

1 - Take the max number of the array so it doesn't explode doing the exponentiation.\
2 - Before the exponentiation to the array subtract the max value of the array to the array ,then exponentiate it.\
3 - The Result is just the array exponentiated divided the summatory of the array exponentiated.

Btw Softmax is mostly applied in the last dimension so the axis = -1 should be a standard and is how I did it in my Minitorch.\
(Softmaxx with double "xx" is on purpose to make sure that it doesn't do a conflict with the Softmax function of my minitorch).

In [ ]:
def Softmaxx(inputs):

    max_num = np.max(inputs, axis = -1, keepdims=True)

    scores = np.exp(inputs - max_num)

    out = scores / np.sum(scores, axis = -1, keepdims=True)

    return out

And I'll do its backpropagation that is something that I also did in my Minitorch.\
Its just using the trick with the formula of the Jacobian.

I'll not go deep into mathematical reasons but the derivative of Softmax is just the out of the foward pass of the softmax multiplied by the substraction of the summatory of the gradient multiplied by the out of the foward to the gradient.

In [ ]:
def backprop_Softmax(out,gradient):

    summatory_gradout = np.sum(gradient * out, axis = -1, keepdims=True)

    dSoftmax = out * (gradient - summatory_gradout)

    return dSoftmax

Here,is important to first of all create our parameters (Query, Key, Value) outside the function.\
To create them we just gonna use the embeddings dimmensions for something that I'll explain in the multi_head_atenttion function.
Like always when you create a tensor with random values you need to multiply them by the Xavier or He init formula(The one that comes better for the reason of what you gonna use that tensor with random values).

Note:The Wo variable is also important and is written in the paper "Atenttion is all you need" is like a "mixxer" of the reponses of the heads to make the output have more coherence,but its responsabilities are not as "important" as the other 3,just for that reason to clarify the panoram I'll treat it different.

Every parameter should be a Tensor of minitorch to be able to train it with Adam.

In [ ]:
Wq = Tensor( np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd), requires_grad = False )
Wk = Tensor( np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd), requires_grad = False )
Wv = Tensor( np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd), requires_grad = False )

Wo = Tensor( np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd), requires_grad = False )

Here is the heart of the transformer,The MHA(Multi Head Atenttion),I've divide it by sections for me to be easier to explain:

Section 0: This is just preparation,we take the shape of our inputs and create the size of every of the 4 heads,size which will be 32 embeddings each head.


Section 1: Here we combine the data of our input with the data of ours parameters just to differentiate them of the function of what each one does.\
Creating the variables Q (query), K(key), V(value).


Section 2: This step is simply to organize them in orden to be able to do its respective function down the road.\
First we take the Q,K,V data from the previous section and reshape them to become a tensor4D(4,8,4,32) and then transpose them to be able to work with the 32 embeddings(1 head) of each one of the token inside the block_size.


Section 3: Here is when we use the differentiation of each one.\
Commonly I think about the query as the "signal" that the token who wants to predict the next one emits to start searching in a library(the "What i want").\
The key I interpret them as Label/Name of the books (each token have 4 books describing it,the heads).\
And finally the value as the content of the book.

Our purpose is create 4 new books(One Token) so in order to create them we going to search in the library and take the content that we want for those next new 4 books.

So in this section we compare every label of the book in the library with what we want to know how much of every book we going to take.\
The formula will be Q*K.transpose for that searching,transpose because (excluding the dimension that we don't care right now) we got Q=(8,32) and K=(8,32) so in orden to do the matmul we transpose the K ->(32,8) and now we can start "searching" (8,32)*(32,8).

Finally for this section, we going to do the scaling to the weights(wei), This is just for the Softmax so it doesn't explodes with numbers to high.\
Like lowering the volume of the values so Softmax doesn't get stunned.


Section 4: We need to do Softmax in order to be able to know how many of each of the heads we need to take to predict the next token correctly.\
For that reason we do Softmax in the last dimension,but something that happens when we do the Softmax is that the Softmax also takes in consideration the next tokens(the ones that we want to predict),for example,if we are in position 5 and we want to predict the token of position 6 we going to do a Softmax to all the tokens from position 0 to position 7 and if you notice that is an issue because with Softmax the model will cheat because it will know already what token is next.\
The solution for that is an Atenttion mask,what is that you will ask,well you can see it as phantom tensor that we put on top of our tensor of tokens,this atenttion mask will have a triangular form that we gonna fill with 1s an the rest with either "-inf" or a number extremely low.\
What this will do?This atenttion mask will prevent the model to cheat because the mask will replace our tensor with numbers in a form that every time we want to predict the next token the values of the next token will be 0 cause of the traingular form of the mask, this will tell Softmax that we don't want to take the any information of future tokens in order to be able to predict it using the past tokens only.\
So in the code we first create the mask using numpy and given the dimensions of our block size (8,8).\
Then using again numpy we can do this multiplication that I've talked about earlier to assing the values of wei where the ones are and the positions without 1 we fill it with negative infinite(in this case I'll use negative 1 billon just to make sure that I don't will fine any computer error down the road,which it could be the case if I leave it with negative infinite).


Section 5: Now that we are sure that the model will not cheat we can just do the Softmax.


Section 6: Here because the softmax already tell how many percentage of each head we want to take to have a good prediction we just do a matmul between how many of the values we want and the respective values of each head.


Section 7: Now that we got our values we gonna transpose and reshape them in order to have the same "form" of the tensor x that was our input to the function,to make things easier for future functions that it going to pass through.


Section 8: Finally and now that we got our results of the multihead atenttion we can "mix them".\
Every head you can see them as a proffesional in differents aspects of the tokens,one head can be an proffesional in semantic,other head and professional in sustantives,etc.\
So our output tensor will be just a concatenation of different proffesional opinions of what will be the next token.\
The heads are independent so the concatenation will sound like different peoples doing the same project and just combining they parts to the same project.This matmul will prevent that,it will "mix" the results (the opinion of each proffesional) to make the final output sound more coherent.


Section 9: While doing the backpropagation of the MHA I learned that I'll need the cache of the MHA ,so I put this section later in the creation of the model because without this information that I saved in the cache variable I can't do the backpropagation of the MHA.\
Is not the first time that I save a cache variable,for example in the CNN from scratch in the maxpool function you need to save the cache to know how was the size before to make the tensor of the gradient become a tensor with the correct size and dimensions.

In [1]:
def multi_head_attention(x, Wq, Wk, Wv, Wo, heads):

    # Section 0
    
    batch_size, block_size, total_embeddings = x.shape # The classic (B, T, C)

    head_size = total_embeddings // heads 

    
    # Section 1
    
    Q = x @ Wq

    K = x @ Wk

    V = x @ Wv

    
    # Section 2
    
    Q = Q.reshape(batch_size, block_size, heads, head_size).transpose(0, 2, 1, 3)

    K = K.reshape(batch_size, block_size, heads, head_size).transpose(0, 2, 1, 3)

    V = V.reshape(batch_size, block_size, heads, head_size).transpose(0, 2, 1, 3)

    
    # Section 3

    wei = Q @ K.transpose(0, 1, 3, 2)

    wei = wei * (head_size**-0.5) # More fancy than "(1 / np.sqrt(head_size))" which is also valid.


    
    # Section 4

    mask = np.tril(np.ones((block_size, block_size)))

    wei = np.where(mask == 0, -1e9, wei) #Underflow instead of -inf.


    
    # Section 5 

    probs = Softmaxx(wei)



    # Section 6

    out_heads = probs @ V


    
    # Section 7

    out_concat = out_heads.transpose(0, 2, 1, 3).reshape(batch_size, block_size, total_embeddings)


    
    # Section 8

    out = out_concat @ Wo



    # Section 9

    cache = (x, Wq, Wk, Wv, Wo, Q, K , V, wei, probs, out_concat, head_size)

    return out,cache

Initialization of the Parameters Gamma and Beta,which we will use in order to save and train them to make a more accurate model.\
The gamma parameter represents the Standard Deviation,which will be a tensor2D full of 1s (128,1),this can't be a tensor2D of zeros because if we multiply the information by 0 obviously the result will be 0 and we "kill the information".The first time it will pass as it is (because of the multiplication by 1)and then the parameter gamma will get actualize in order to pay more attention to the small values and have a more accurate standard deviation.\
The beta parameter represents the ideal mean that we want,it will start as a tensor2D full of 0s with the same size as the gamma tensor(128,1).

Every parameter should be a Tensor of minitorch to be able to train it with Adam.

In [ ]:
gamma1 = Tensor( np.ones((n_embd,)), requires_grad = False )
beta1 = Tensor( np.zeros((n_embd,)), requires_grad = False )

gamma2 = Tensor( np.ones((n_embd,)), requires_grad = False )
beta2 = Tensor( np.zeros((n_embd,)), requires_grad = False )

Now that we have the "opinions" after we pass through the MHA we need to normalize them,because right now we can have for example: a value that is 50 and a value that is 0.01,here what is going to happen is that the higher value (50) will overlap the lower value (0.01).\
This is something that we don't want because both value are usefull,so in order to avoid this we will normalize them using a simple formula which is:\
input normalized is the substraction of the mean of the inputs to the inputs divided by the standard deviation of the inputs(the epsilon is as always just for the PC to take tiny numbers and don't convert them to 0 which is what the PC tends to do).

You can see x - x_mean like the process of making the numbers go lower, this a type of normalization like we do in the Softmax function but here instead of the Maximum number we use the Mean number.\
Then dividing with the standard deviation is the process to make them be "closer to each other",for example if they were scattered from 0 to 500 this will make them be scattered from 0 to 1.\
This is something that you will find in every layer norm function because is something standard in all of them,the things that changes in every model tho is the gamma and beta parameters because is something that is trained as every weight in the model,changing the specific deviation and mean in each model.(We need to save the cache for the manual backpropagation of the layer norm where we gonna give to gamma and beta their respective gradients)


PD: while I was learning about the history of this I come across something.\
The LN(Layer Norm)previously when the paper "Attention is all you need" came out,the LN was only used in the post-MHA,now in modern times the LN is used pre-MHA,models like GPT-2,GPT-3,Llama use the LN before the MHA,this is because doing it before the MHA makes the model easier to train(more stable).\
This doesn't change the explanation neither the formula but it changes what values the function takes and when you gonna use it in the training pipeline.

In [ ]:
def layer_norm(x,gamma,betta, eps=1e-5):

    mean = x.mean(axis = -1, keepdims=True)

    var = x.var(axis = -1, keepdims=True)

    std = np.sqrt(var + eps)

    
    
    x_hat = (x - mean) / std


    out = gamma * x_hat + beta


    cache = (x_hat,std,gamma)# All in the same variable,later we will unpack it.

    
    return out,cache

This is just a "Residual Connection",our inputs (x) will travel across the entire transformer and after almost every function we gonna call this function that is just a addition of the previous inputs with the output of the function that have used those inputs.\
This will help with the gradient vanish cause by the multiplicatiion and all this function.\
As an anology you can imagine a signal(the inputs "x") that need to travel across 12 floors of a building(the transformer) at the time that the signal have reach the 12th floor is almost inexistent because in every floor it gets weaken,so to avoid this we enrich the signal in every floor to be able to reach the top of the building.



This function is so extra I know,but is for me to learn it and to have a more clean "memory map" of what is happening in the model.

In [ ]:
def residual_connection(x,functionout_x):
    return x + functionout_x

Here we initialize our parameters for the Feed_Foward Network(FFN):\
To create the weights we create a tensor2D of random values(W1) which will have the amount of rows of the number of embeddings of the tensor that comes from below (128 embds)and the amount of neurons that we want (512 neurons(128 * 4)).\
Why "*4"? Because that's the sweet spot that the creators found in the "Attention is all you need" paper to do this expansion and compression in the FFN.

For W2 we need to do the same but with the opossite values for the "columns & rows" of the tensor2D of the hidden layer of our FFN,comparison:\
W1 = (128,512)\
W2 = (512,128)\
Then we just use again another activation formula,and in this case I'll use He init for the hidden layer weigths because it is better for doing ReLU.\
And Xavier for the output layer weigths because after we do the residual connection using the output of the FFN function we going to do a Softmax and for Softmax is better(actually just standard idk if is better at this point) Xavier for it.

Then for the Biases as we know we need a Bias per Neuron so that what we do,we create a tensor1D of (1,512) for the B1.\
And a Tensor1D of (1,128) for the B2.

Every parameter should be a Tensor of minitorch to be able to train it with Adam.

In [ ]:
W1 = Tensor( np.random.randn(n_embd,n_embd * 4) * np.sqrt(2 / n_embd), requires_grad = True ) 
W2 = Tensor( np.random.randn(n_embd * 4,n_embd) * np.sqrt(1 / (n_embd * 4)), requires_grad = True )
B1 = Tensor( np.zeros((1, n_embd * 4)), requires_grad = True )
B2 = Tensor( np.zeros((1, n_embd)), requires_grad = True )

Here we have the Feed-Foward Network, in comparison we can see this part as the Neural Network of MNIST.\
What we have here is the Input that comes from below (Pos_encoding,MHA,LayerNorm,Residual_Connection) going to the hidden layer and "mixxing" with the W1 & B1,this creates Z1.\
Then Z1 does a ReLU and we get the activation of Z1 which is A1.\
Then A1 can do the same that previously which is "mixxing" but now with W2 & B2 resulting in Z2.\
Finally we return the values and if you notice it we don't do the A2 that we normally have to do, because before that we need to do the residual connection of this output of the function to the input of this function to just then get the A2, which will be Z2 enriched by the inputs and the result pass it through Softmax.

For a more In-Deep Explanation of what is happening in this I encourage you to see my Neural Network from Scratch repository where I explain this better.

In [ ]:
def FFN(x,W1,B1,W2,B2):

    Z1 = x @ W1 + B1
    
    A1 = Z1.ReLU()

    Z2 = A1 @ W2 + B2

    return Z1,A1,Z2

Preparation of the final weights which we will use in the linear head function.\
its activation is the same as the parameters of the FFN and we gonna use Xavier for them because the next step is softmax.

In [ ]:
W_final = Tensor( np.random.randn(n_embd,vocab_size) * np.sqrt(1/n_embd), requires_grad = True ) 
B_final = Tensor( np.zeros((1,vocab_size)), requires_grad = True )

Here comes a simple function,the Linear head.\
Because right now we have the tensor3D Z2(4,8,128) we can't "read it" (is in machine language right now) we need to traslate it to a human vocabulary of 65 characters (THIS IS NOT LIKE A DECODER & ENCODER, it's just analogies).Like convert the sustantives in the actual words.\
To do that we just need to add separately another layer (the linear head) which will have its own weights and bias to transform the result of the FFN into the logits that we gonna use to calculate the loss and predict the result.

In [ ]:
def linear_head(Z2, w_final, b_final):

    out = Z2 @ w_final + b_final

    return out    

Here we go with the Cross Entropy Loss function,this will tell us how far of the correct result was the prediction of our model.\
First we gonna take each dimension of the inputs,because the inputs are a tensor of my minitorch and I forgot about to introduce the .shape commonly used in numpy to take the shapes,my minitorch can't take the shapes,the solution is to take the np.array (the .data of the tensor) and to that take its shape.

The next step,because we have the Z2 "enriched" with the residual (that is the inputs tensor) we need to take the probabilities of each prediction applying to them a Softmax(the Softmax that I will use for this is the one of my minitorch,keep it in mind).\
To continue the process of getting the loss we need to flat them,the probs because its a tensor of my minitorch will use the Reshape(Capital R)of my minitorch and the targets because they are a numpy array will use the reshape of numpy.

Next step is doing the One Hot encoding ,first we create a tensor of zeros using the same dimensions and size as the probs tensor,then we do a fancy indexing to put inside that tensor of zeros a 1.0 in the correct answer.\
Lastly we convert the tensor mask already one hot encoded in a Tensor of my minitorch to be able to compare it with the predictions tensor.

Then we take the correct probs that is a simply comparison between the tensor of prediction with the tensor of the one hot and this will "turn off" the incorrect answers (it will multiply them by 0 and the correct by 1).\
The eps variable is just for numeric stability for the computer,so it doesn't assume numbers too small as 0.\
Finally we apply the formula of Cross Entropy,what we want is "amplify" the error to do a better correction when we do the backprop.We apply to the correct_probs the epsilon variable and then we do a log to them this will give us numbers that with a small change thanks to the log it amplify the change, .Sum() is to sum all the logaritmics errors of the batch.Because all the logaritmics values of numbers below 1 are negative numbers the next step is to multiply them by -1.0 and divide them by the numbers of predictions because we need to take a mean of them so we don't depend of the size of the batch.\
Finally we return the result(Loss).

In [3]:
def cross_entropy_loss(inputs,targets):

    batch_size,block_size,classes = inputs.data.shape


    probs = inputs.Softmax()
    
    probs_flat = probs.Reshape(batch_size * block_size,classes)

    targets_flat = targets.reshape(-1)
    


    
    oh_mask = np.zeros((batch_size * block_size,classes))

    oh_mask[np.arange(batch_size * block_size),targets_flat.astype(int)] = 1.0 #Fancy indexing for the One Hot

    targets_oh = Tensor(oh_mask, requires_grad = False)


    
    correct_probs = (probs_flat * targets_oh).Sum(axis = 1)

    eps = 1e-9
    loss = (correct_probs + eps).Log().Sum() * (-1.0 / (batch_size * block_size))

    
    return loss 

Again create a function for this is actually not needed,but to learn how is the process that the model have to do I'll create this function and call it when needed.

Because the derivative of the addition of x + y is 1 the gradient just get distributed to both of them.\
Like in the foward pass the residual connection have the same functionality which is "amplify the signal" in order to avoid the gradient vanish,because without the residual connection amplifying the signal if the backprop of the FFN (for example) pass the gradient to the backprop of the layer norm if some of the weights/data are too low, the backprop of the layer norm will make it lower and so on; Pushing the gradient to become 0 and avoiding the correct backpropagation making us that we can't train the model because the gradient ended up as nothing mid backpropagation.

The if statements is just in case any of the variable comes as a "None" and because this is a pretty simple function we can get away just by doing this.

In [ ]:
def backprop_residual_connection(grad_input,grad_output):

    if grad_input is None: 
        return grad_output
        
    if grad_output is None: 
        return grad_input
        
    return grad_input + grad_output    

This is the backpropagation of the layer norm.\
We will take the gradient that comes from above and the cache that we have store earlier in the layer norm function of the foward pass.\
Next step is take the variables stored inside the cache variable and the shape of the gradient.

if you see the mathematical operation that we did in the foward pass to add the gamma and beta parameters in the layer normalization you will see that the operation is similar to a layer in a NN (X @ W + B) but in this case was (X * gamma + beta) so,in therms of beta is pretty simple,to get its derivative you just need to do the same thing that you do to get the derivative of a bias,which is the summatory of all the error.\
Then the derivative of gamma is conceptually similar,the difference is that in the NN you get a dot product and here you do a multiplication element by element,so the derivative will be the summatory of the comparison of the gradient with the x_hat.

Now dx is similar to get A1 in a NN the loss multiplied by the weights(in this case the parameter gamma).\
Next is "dx" which is the derivative to the input of the layer norm and honestly is just a mathematical formula that we traslate to code.If you want to know more about the formula you can check it out in this link: "https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html" (copy and paste it in the browser without the quotes if it doesn't work).\
I'll not explain the formula because is pure mathematical logic,but to understand what is happening without the need of the link you can imagine that because is the layer_norm a process in the middle of the transformer it will have a previous function and in order to continue the backpropagation we need to keep passing the gradient and thanks to that formula the gradient can reach the input of the function and continue to the next backprop_function.

In [ ]:
def backprop_LayerNorm(grad_output,cache):

    x_hat, std, gamma = cache

    batch_size, block_size, n_embd = grad_output.shape


    dbeta = np.sum(grad_output, axis=(0,1))

    dgamma = np.sum(grad_output * x_hat, axis=(0,1))



    dx_hat = grad_output * gamma

    dx = (1.0 / n_embd) * (1.0 / std) * (
        n_embd * dx_hat -
        np.sum(dx_hat, axis = -1, keepdims = True) -
        x_hat * np.sum(dx_hat * x_hat, axis = -1, keepdims = True)
    )


    return dx, dbeta, dgamma

This is the "Final Boss of the Backpropagation"(honestly I felt more difficult the backprop of the layernorm because of its mathematics in dOut,this is practically just a "dance" of dimensions).\
Because of my previous creation of the Autograd engine, I learned a lot of the back propagation of the different formulas,here we use  in a huge part the bakpropagation of the matmul which is:\
if Z = X @ Y \
The derivative of X is = grad_output @ Y.T \
The derivative of Y is = X.T @ grad_output\
If you want to memorize this is pretty simple, just put your tensor of gradients in the same place of the foward of the variable that you want the derivative and multiply it by the other variable transpose.

And Again,I'll divide the function in sections to make it easier to explain,the order of the MHA_backprop is basically a 9 to 1 of the foward pass:

Section 1: Here we just take our variable and prepare it to use everything that we saved in the foward pass.

Section 2: First we want the derivative of Wo which we will train later on in the training loop, to get the derivate,as I told you previously we gonna use the formula of the backward of a matmul,doing a transpose of the dimensions (original dimensions: batch_size, block_size, n_embd) -> (new dimensions: batch_size, n_embd, block_size) we prepare the terrain to make a reshape using "-1", which doesn't mean "grabbing the last index" (like in a list),it means "grabbing every that is left over" and because then we gonna do a dot product matmul we need "the dimension on the right" of the new tensor2D of the same size as the "dimension of the left" of the gradient tensor that will be block_size,for that reason the reshape is(-1, block_size).\
Now the dout_concat is much simplier because we gonna use the tensors2D as they are,so we apply the formula and we get the dout_concat.


Section 3: As we have concatenated the "responses" of each of the 4 proffesionals(heads) we need to undo that right now,for that reason we use the same dimensions and size as we had in the foward pass,making the a tensor4D (batch_size, block_size, heads, head_size) we will use "-1" instead of head_size to avoid any future dimensional problem and as a good practice for future projects.\
Then we just need to transpose it to have again the same order of dimensions that we previously had in the foward pass.

Section 4: Now that we have the gradient in the dout_heads we can use it like we have used the grad_output of the previous matmul,following the same mathematical formula and getting the derivative of the values per head,which is a requisite to get the derivative of the parameter Wv.\
Next we gonna get the derivative in respect of the probabilities of the foward pass,if the foward pass was (out_heads = probs @ V) the derivative of the probabilities will be (dprobs = gradient @ V.T).

Section 5: Next step is to get the derivative of the Softmax,for that we gonna use the trick in the Jacobian,which is the following formula:\
dSoftmax = out * ( gradient  -  summatory( gradient * out ) )\
Where "out" is the out of the softmax in the foward pass (probs), and gradient is the one that comes from above(dprobs).

Section 6: This section I interprete it as something similar to a derivative of ReLU (It's not the same but it make it easier to understand this way).\
The gradiente where we put the -inf (-1e9 to avoid issues) is 0.So we create another mask and we gonna do the same process that we did in the foward pass again.\
Once we pass it through the mask that we just created the remaining is getting the actual derivative of the weigths which is just multiply it by the 1 / np.sqrt(head_size) or how we did in the function(which means the same mahematically speaking) in the foward pass for the sake of the chain rule and be able to continue backproping.

Section 7: In this section we gonna have a heavy "dance of dimensions" because in the foward pass we did the search (Q @ K.T) and there is the complication, K for being transpose will change how the backward of the matmul works.\
We want the derivative of the Q and K heads (like the dV_heads variable) and we have the K already transpose for the foward pass,so to get the dQ_heads we do (dwei @ K without transposing the K again).\
Okey now that we got dQ_heads here is where it gets more complicated.To get the dK_heads we do (dwei.T @ Q),originally the formula standard should be (Q.T @ dwei),but in this case we specifically change that because of the K tranposed in the foward pass.The difference is that if we do the standard formula we gonna get (dK_heads.T) and we need to tranpose it again,is an unnecessary step that we can avoid just using the properties of the matrix and for that reason it will end up in the formula that we have previously mentioned (dwei.T @ Q).

Section 8: In this step we gonna collapse the dimensions,leaving us with tensors3D instead of tensor4D and "eliminating the heads".

Section 9: At the beginnig (in the foward pass of the MHA) we did something like mixxing the parameters with the input (x) this resulting in the variables Q,K and V.Now we gonna use the standard backward of the matmul to get the derivative of each one of the Parameters(Wq, Wk, Wv).\
First we need to do the reshaping and transposing of the input and storage it in a variable so we can use it in the next lines.\
Using the formula we do (dParameter = X.T @ gradient_of_the_parameter) and adding a reshape in there because the original parameter was a tensor2D and we want to matmul 2 tensors2D in order to do it.The dimensions are kinda difficult to explain in writting,so I'll rather not try it because it can cause more confusion than understanding of the function.

Section 10: Finally we have the derivative of the input of the MHA,which following the chain rule is just ( dParameter_raw @ Parameter )this is something similar to get the A1 in the NN of MNIST,but here we have more parameter to consider so we just apply this chain rule to all of them and then we sum them,leaving us with the derivative of the input.

Lastly we return the dx to keep backproping and the derivative of the parameters to actualize them.

In [ ]:
def backprop_MHA(grad_output, cache):

    # Section 1
    
    x, Wq, Wk, Wv, Wo, Q, K, V, wei, probs, out_concat, head_size = cache

    batch_size, block_size, n_embd = x.shape

    heads_amount = Q.shape[1]


    # Section 2

    dWo = out_concat.transpose(0, 2, 1).reshape(-1, block_size) @ grad_output.reshape(-1, n_embd)

    dout_concat = grad_output @ Wo.T



    # Section 3

    dout_heads = dout_concat.reshape(batch_size, block_size, heads_amount, -1).transpose(0, 2, 1, 3)



    # Section 4

    dV_heads = probs.transpose(0, 1, 3, 2) @ dout_heads

    dprobs = dout_heads @ V.transpose(0, 1 ,3 ,2)


    
    # Section 5

    d_wei_masked = probs * (dprobs - np.sum(dprobs * probs, axis = -1, keepdims = True ))



    # Section 6

    mask = np.tril(np.ones(( block_size, block_size )) )

    d_wei_scaled = np.where(mask == 0, 0, d_wei_masked)


    dwei = d_wei_scaled * (head_size**-0.5)


    
    # Section 7

    dQ_heads = dwei @ K

    dK_heads = dwei.transpose(0, 1, 3, 2) @ Q



    # Section 8

    dQ_raw = dQ_heads.transpose(0, 2, 1, 3).reshape(batch_size, block_size, -1)

    dK_raw = dK_heads.transpose(0, 2, 1, 3).reshape(batch_size, block_size, -1)

    dV_raw = dV_heads.transpose(0, 2, 1, 3).reshape(batch_size, block_size, -1)



    #Section 9

    x_flat_T = x.transpose(2, 0 , 1).reshape(n_embd, -1)
    
    
    dWq = x_flat_T @ dQ_raw.reshape(-1, n_embd)

    dWk = x_flat_T @ dK_raw.reshape(-1, n_embd)

    dWv = x_flat_T @ dV_raw.reshape(-1, n_embd)


    # Section 10

    dx = dQ_raw @ Wq.T + dK_raw @ Wk.T + dV_raw @ Wv.T
    
    
    return dx, dWq, dWk, dWv, dWo

Here is the backpropagation of the positional encoding.\
In the foward pass we don't apply the pos_table with a matmul,but instead we have used indexing and addition so the backward will notbe the same as the backward of a matmul but a summatory of all the error.\
Why we select the axis 0(the batch dimension)? because if for example, in the position 5 we have an error in any part of the batch, it will get added to the sum of a total error and this total error will tell the parameter how much it needs to change.\
Once saved the derivative of the positional encoding we return it.

In [ ]:
def backprop_positional_encoding(grad_output):
    
    dWPE = np.sum(grad_output, axis = 0)
    
    return dWPE

This backprop is a lil bit more complex because the same token could have appear in other part of the same batch and if we don't consider this difference they will overlap each other gradient.\
This function will take the gradient, the original input (xb) and the original token_table parameter,but we don't gonna use the original parameter per se. Instead we gonna use the form of this parameter to create a new tensor2D with the same form and fill it with zeros to be able to do a selective accumulation.\
Then it comes the time to do the np.add.it which will add in the correct position the gradient of each token no matter if they repeat themself, for that reason we use the np.add.it,without it if we do the common indexing like we did in the foward pass they will overlap each other.\
Lastly we just return the derivative of the parameter.

In [ ]:
def backprop_token_embd(grad_output,xb,token_table):

    dWTE = np.zeros_like(token_table)

    np.add.at(dWTE, xb, grad_output)
    
    return dWTE

Here we gonna use the Adaptative momentum(Adam) optimizer that's inside my minitorch.\
Creating the variable of params which will be a list of all the parameters that we have created in the entire transformer.Then initializing them inside our class Adam of the optim folder inside the minitorch to be able to do the .step() and the .zero_grad().\
I also saw people talking about the "sweet spot"/"golden standard"(karpathy also used it like this in his tiny transformer) being  a learning rate of 3e-4 (0.0003),so I gonna try it myself and see how that works here.By default the learning rate inside the Class Adam in my minitorch is 1e-3 (0.001).

In [ ]:
params = [token_table, pos_table, Wq, Wk, Wv, Wo, gamma1, beta1, gamma2, beta2, W1, B1, W2, B2, W_final, B_final]


optimizer = optim.Adam(params, lr=3e-4)

The next is the preparation of the variables needed for the training loop which is the next step in the project.

max_iters: By the naturality of the transformer we don't use epochs like in previous projects (NN & CNN) which tell us the amount of times that the model travel the entire dataset.We use interval because the transformer takes small portions of the dataset and because we do this cutting of small portions of the dataset the amount of posibilities goes to infinity so is kinda dumb think about it like epoch,instead we gonna think about it like iterations (the amount of actualization of the parameters).

eval_interval: this is the variable that tell us every how many iterations we evaluate our parameters.

eval_iters: this variable works with the "eval_intervals" variable,because this variable will tell us the amount of iterations that we need to take to measure the loss,because the "eval_intervals" variable tell us where to stop and the "eval_iters" variable of many iterations we take.

In this transformer we gonna use a dictionary to save the results of the loss history.\
The remaining variables are just the standard early stopping variables that I'll will use in the final part of the training loop.


The last part is just to create a variable to make the training easier,using "tqdm" which will show us information of the current process, velocity,estimated time of arrival and loss.

In [ ]:
max_iters = 5000
eval_interval = 500
eval_iters = 200

loss_history = { 
                 'train':[],
                 'test':[] 
               }

best_loss = float('inf')
patience = 5
wait = 0


progress_bar = tqdm( range(max_iters), desc= "Shut it off, Otto, shut it off!" )

Here we create a new file called checkpoint if it isn't created yet.\
The we create a function that we gonna call later on in case we want to save the current progress that we have or for some issue that we could encounter along the way.\
First we create a dictionary which gonna store the parameters (only their data) in order to  use that dictionary in the next part which is the actual checkpoint,where we gonna store the number of iterations that we have done, the parameters previously mentioned, the optimizer that as you can see is "special", why? you may ask, is because Adam use momentum in they variables and we need to save the momentum because normally when a kernel have a failure (its state becomes "unknown") you have to restart it, once you have restarted it it will delete the momentum leading to a break in the training because of convergency errors.\
Then we save the history of the loss (loss_history).

Lastly using pickle which is a library that will help us to store the file that we have created with the checkpoint inside the checkpoint file,giving to every save_checkpoint the name according to its number of iterations in the case that we don't pass a filename. 

In [ ]:

#--Creation of file---#

if not os.path.exists("checkpoints"):

    os.makedirs("checkpoints")



#-- Actual Checkpoint---#

def save_checkpoints(iters, optimizer, history, filename = None):

    params_data_only = {
        "token_table": token_table.data, "pos_table": pos_table.data,
        "Wq": Wq.data, "Wk": Wk.data, "Wv": Wv.data,"Wo": Wo.data,
        "gamma1": gamma1.data, "beta1": beta1.data, "gamma2": gamma2.data, "beta2": beta2.data,
        "W1": W1.data, "B1": B1.data, "W2": W2.data, "B2": B2.data, 
        "W_final": W_final.data, "B_final": B_final.data        
    }
    
    checkpoint = {
        "iters": iters,
        "params": params_data_only,
        "optimizer": optimizer.__dict__,
        "history": history
        }

    if filename is None:

        filename = f"checkpoints/model_iters{iters}.pkl"
        
    with open( filename, "wb" ) as f:

        pickle.dump(checkpoint, f)

This is something that we will regulary do,after every 500 iterations (our eval_interval) we will call this function to run a test and give us a result which will be a mean of the loss of the test_batch.

As you can see this is just a simple foward pass (only the foward pass),runnning the test_dataset instead of the training passing the string "test" to the get_batches function ,which will give us the inputs(xb) and the targets(yb).\
Next what we gonna do is travel through all the function in order using the parameters already trained by a certain amount of iterations(I'll explain this more in detail in the training cell).\
Finally we gonna return the mean of the loss of this single test_batch that we did.

In [ ]:
def estimate_loss_test(batch_size, block_size):

    test_losses = []

    for _ in range (eval_iters):

        xb, yb = get_batches("test", batch_size, block_size)

        # Only Foward

        x_token_and_pos = positional_encoding(xb, token_table.data, pos_table.data)

        
        residual0 = x_token_and_pos

        
        x_ln1 = layer_norm(x_token_and_pos, gamma1.data, beta1.data)

        out_mha = multi_head_attention(x_ln1, Wq.data, Wk.data, Wv.data, Wo.data, heads = 4)

        x_mha = residual_connection(out_mha, residual0)



        residual1 = x_mha



        x_ln2 = layer_norm(x_mha, gamma2.data, beta2.data)

        _, _, Z2 = FFN(x_ln2, W1, B1, W2, B2)

        residual2 = residual_connection(Z2.data, residual1)


        logits = linear_head(residual2, W_final, B_final)

        loss = cross_entropy_loss(logits, yb)


        test_losses.append(loss.data)

    return np.mean(test_losses) 

This function is the cool part of the transformer, it will use the parameters trained by the model and output an arbitrary number of tokens(50 by default I set) that later we are going to decode and see how the model " Talks ".

*There is a option that I have of start the sampling with the letter R of "Romeo" for example, but honestly,I've been playing with this a long time and first I want to see it learning, so for now I'll use a np.zeros instead of the np.array(encode("R")) and that*

So first is the creation of the variable idx which is the seed to start the sampling(which gonna be 0).

Next declaring a loop to ve able to predict the amount of token that we wanted.\
inside that loop is the context windows variable that we can get indexing the seed, the seed is a tensor2D so we take the entire first dimension which is the batch dimension and then we just gonna have a context window of 8 tokens (-8) starting taking from that to the final of that dimension.\

After all of that we just gonna do a foward pass of the transformer(in the training loop I'll explain everything about the foward pass).\
Until the logits variable all is pretty standard,but the next step is more specific of the sampling, so we take the last token of the dimension of time (block_size) in order to predict the next one.\
Next we apply our manual softmaxx (2 "x") function to get the probabilities of the next token.\
Next we let Numpy give us "the choice" passing the vocab_size and then the probabilities (p= probs) of the choice that we want of NumPy according to our softmax previously applied.\
Then we just concatenate the tokens that we got to make the sentences to our tensor of zeros "idx".\
Finally returning the tensor2D that we got but using the indexing 0 to flatten it so we return just a numpy array of numbers instead of a tensor2D.

In [ ]:
def generate(max_new_tokens = 50):

    idx = np.zeros((1,1), dtype = np.int64)

    for _,_ in range (max_new_tokens):
        
        ctx_win = idx[:, -8:]

        #Foward#
        x = positional_encoding(ctx_win, token_table.data, pos_table.data)

        residual0 = x

        x_ln1 = layer_norm(x, gamma1.data, beta1.data)

        out_mha = multi_head_attention(x_ln1, Wq.data, Wk.data, Wv.data, Wo.data, heads = 4)

        residual1 = residual_connection(residual0, out_mha)

        x_ln2 = layer_norm(residual1, gamma2.data, beta2.data)

        _, _, Z2 = FFN(x_ln2, W1, B1, W2, B2)

        residual2 = residual_connection(residual1, Z2)

        logits = linear_head(residual2, W_final, B_final)
        #-------#
        

        last_logits = logits[:, -1, :] # (1, vocab_size)


        probs = Softmaxx(last_logits)

        next_token = np.random.choice(len(probs[0]), p = probs[0])

        idx = np.concatenate((idx, [[next_token]]), axis = 1)

    return idx[0]

Here is finally the training of the transformer which is a single loop of iterations(I'll explain what I mean by this).


TRAINING LOOP:


1) <u>Foward Pass</u>.

- In this loop we will continuously calling the get batches function and it will give us the inputs(xb) and the targets(yb) of the training dataset(because we pass to the function the string "train")the inputs and targets will have a batch_size of 4 and a block_size of 8 meaning that our inputs and targets will be tensors2D.

We can divide the transformer in parts to make it easier to understand,some parts have their own "Main" function and their "Accessories" functions:

-Token Embedding & Positional Encoding.

-Multi-Head Attention & Layer Norm(this being the "accessory").

-Feed Foward Network & Linear Head(this being the "accessory").

-Cross Entropy Loss Function.



- Now,the tokenization and the positional encoding(they are together in a single function), we will past our inputs and the parameters token_table and the pos_table but only their .data(because they are tensor of my minitorch),that data is what we will train.

- What comes next is something that we will use along the foward pass and we gonna use in the backpropagation too and is the residual connection,something that I've explained twice when doing the actual function and its backpropagation which aren't needed (because is just a "x + y")but I wanted to do it to have a better mental map of the architecture of the transformer.\
We gonna use them as checkpoints(inputs save) to connect the differents "parts" of the transformer that I've previously mentioned(residual connection is like glue and an amplifier at the same time).\
Is important to have in mind that we have 3 saving inputs but we only do 2 residual connections (at first I was confused and the model didn't work because of this issue of nomenclature, you can't see the difference right now because I've change it but is something to have in mind if you come across to something similar).

- Once we did that the next part is the brain of our transformer (the MHA)what we gonna do is a pre-MHA and post-MHA layer norm model,what this means is that the layer norm will be applied before and after the passing through the multi-head attention function,so in order to achieve that we call the layer norm function for the first time passing the result of the previous part of the transformer (you can also use the "residual0" variable but we gonna use the x_token_and_pos variable instead to make it more clear) and the .data of the parameters used in the layer norm (gamma1 & beta1, important the "1" in the name because this parameters are completely different from the post-MHA).That will result in a variable which are our inputs normalized and the second variable which is the cache(something that we gonna use in the back propagation).

- Then is the turn of the "Main" function once the inputs already passed the "accessory" function (layer norm),the MHA function will take those inputs and the .data of the parameters Wq,Wk,Wv,Wo.This will result in a variable called "out_mha" which are inputs procceced by the MHA and a variable called cache_mha which we will use in its back propagation.

- Because we passed through the MHA we've lost "signal strength" and now we need to enhance it to keep the foward pass,for this reason we take our previously saved signal (residual0) and we combine it with the output of the MHA.Creating the variable x_mha.

- What comes next is the second input save (residual1), this checkpoint is a copy of the x_mha variable previously mentioned.

- Following up is the second layer norm, using the same variable (x_mha) but now passing differents parameters (gamma2 and beta2,only their data), this is the post-MHA layer norm and it will return us the x_ln2 and the cache_ln2 variables.

- Here, in the next mini_section is where the transformer becomes "hybrid", because we're goint to convert the inputs(x_ln2) and the residual connection(residual1) into tensors of my minitorch,the reason for this is that I'm planning to use my minitorch to do an autograd in the parts of the transformer that I already know,for example the FFN which I find it basically the same as the Neural Network from the dataset of MNIST.\
An important part of this "Tensorification" is the declaration of the "requires_grad = True" which will provide us with their respective gradients to continue doing the back propagation manually after we do the autograd ( the classic "loss.backward()" ).For now on to the functions we will pass the entire tensor not just their .data this is because for the minitorch to do the autograd we will need the entire tensor with all their context,gradient requirements and data for it to work.

- In this next part we are already using the minitorch to do everything for us , it will save the parents of every tensor created by mathematical operations (._ctx) making possible to do the autograd in the future.\
Using the x_ln2 tensor already converted in a tensor of minitorch we pass it to the function Feed Foward Network(FFN) along with the parameters necessary to run that function(W1, B1, W2, B2).This will return the variables Z1,A1,Z2.


-  Using the variable Z2 (which is the one that we usually do softmax in a normal NN to get the prediction) we enhance it with the last residual connection that we've saved (residual1).This creating the variable x_ffn(residual2).


- Now we need to convert it into something that we can read (one of the 65 characters in our vocabulary),for that we just add another layer similar to the ones inside the FFN,this layer its called linear head and we will pass the parameters needed for that function (w_final & b_final).

- Next is the last part of the transformer which is the cross_entropy_loss function where the output of the linear_head function will go.Inside the loss function is a softmax to get the probabilities and then use those probabilities and compare them with the targets to measure how wrong the predictions are,then intensify those errors with the loss function (the log)and finally return the mean of the errors which will be stored in the variable loss.



2) <u>Loss Storing</u>.

- Here we just gonna store the .data of the variable loss inside our dictionary where we store the progress made by the training of the model,this will be store inside the key value "train" for us to later make a graph of the loss.



3) <u>Back Propagation</u>.

- The first step is to use our autograd engine to make the work for us.So, we call the .backward() on our last tensor (loss) it will backpropagate automatically throught the cross_entropy_loss, linear_head, residual_connection and FFN functions while filling the grad instance of every tensor that we use after the tensorification in the foward pass.

- Now that we have the gradient of the initial tensors that we have created in the tensorification we will take the .grad of those tensors (the gradient) and we pass them to the backprop_residual_connection which is just a sum of the gradient of the residual2 and the gradient of the x_ln2 but I wanted to created that function to make it easier to follow the pipeline and explain what everything means.This will give us the classic "grad_output" the gradient that comes from "above" in any neural network as a numpy array and once its in numpy array form we can continue backpropagating manually ourselves.Always the backpropagation of the residual connection will be "jumpying" from one residual connection to other residual connnection,because the path will be bifurcated, one will be doing the normal and "slow" route doing the backpropagation of every function(the x_ln2 in this case) and the other (the residual connection) is the one that will be jumpying, finally they will meet again in every "checkpoint"(where we saved inputs in the foward pass).


- The next step is that we take our numpy array of gradients that we adquired doing the backpropagation of the residual connection previously and pass it to the backpropagation of the layer norm function at the same time we need to pass the cache that we saved in the foward pass.This will give us the gradient of the input of the second layer norm and the gradient of the parameters,the parameters are tensor of my minitorch to be able to use Adam so in order to use the Adam of the minitorch they need to have a gradient in the .grad instance so we fill the .grad with the gradient that the backpropagation of the layer norm give us for those parameters.


- Once we have that is the turn to do the same but with the mha.So we call the backpropagation of the Multi-Head Atttention and we pass the gradient(dout_ln2) and the cache that we saved in the foward pass(cache_mha) this will give us the gradient of the inputs to keep backproping and the gradient of each parameters used in the foward pass for the MHA (Wq, Wk, Wv, Wo), obviously we will store the gradients of the parameters on each .grad instance of each parameter.


- Now again we need to do another backpropagation of the layer norm,because in the foward pass we have used a model pre-MHA and post-MHA layer norm so the next step will be call the backpropagation of the layer norm and pass to it the gradient (dout_MHA) and the cache_ln1.This resulting in the derivative of the inputs (dout_ln1) and the derivative/gradient of the parameters of the first layer norm (gamma1, beta1) storing them in their .grad instance.

- Here comes another backpropagation of the residual connection using the derivative of the input of the layer norm part(dout_ln2) that will jump to here and the gradient of the input of the last function that we did (dout_ln1) this will result in another tensor of gradients (dout_resconn1)

- Inside the function positional_encoding we did an token embedding because we can do it "together" as you saw in the creation of the positional_encoding in the beggining of the transformer,inside that function in the foward pass the order was token_embedding -> positional_encoding,so for the backpropagation the order of the functions(which will be separed)is positional_encoding -> token_embedding (in this case, because we gonna use the positional learning which will we a trainable parameter) so starting with the backpropagation of the positional encoding we are going to pass the gradient that comes from "above" (dout_resconn1) and that will give us the .grad instance of the pos_table parameter that we have used in the foward pass.


- For the next and final step of the backpropagation we got the back propagation of the token_embeddings function which will take the gradient (the gradient that this function will take is the same as the backpropagation of the positional_encoding, dout_resconn1),which we will pass to the function along with the raw inputs (the xb of the beggining of the foward pass) and the .data of the tensor of the parameter token_table,this will return the final derivative/gradient that we wanted which is the .grad of the token_table.


4) <u>Parameters Actualization</u>.

- The final step in the batch loop is the actualization of the parameters which will be simple because we are going to use the minitorch which have the Adam optimizer and because we purposely made all the parameters tensor of the minitorch we can just do a simple optimizer.step() and this will train all the parameters according to their gradients,finally we do the optimizer.zero_grad() to set the instance .grad empty of all the parameters,making them ready to fill them up again.



5) <u>Iters Loop</u>.

- Here we gonna iterate through all the progress_bar variable which contains the amount of iterations that we want(you can check it in the cell where we declared it). each iteration we are creating a new batch_loss list that we are going to fill with the loss that we had in the batch loop.

- Then comes the Batch Loop following up with the average_loss_epoch which is the mean of the entire batch_losses variable loss and putting that appending that average_loss_epoch variable inside the training key value in our loss_history so later we can graph the loss.


6) <u>Loss Testing</u>.

- Here is where the estimate_loss_test function will come in handy,because everytime the number of iterations matches the evaluation interval we gonna use the estimate_loss_test function with the test_dataset that we selected in the beggining of the transformer (using as argument to the function the string "test" inside the estimate_loss_test function) and passing the batch_size and block_size which in this case will be the same as the training of the model.\
Then we append that loss result inside the "test" key value of our history_loss variable.


7) <u>Early Stopping</u>.

- This is a classical in all my neural networks and projects like this,the only modification is that there is a function call to the save_checkpoints function which in every successfull loss evaluation where we got a loss decrease the early stopping will reset and save the checkpoint in the other case if the model doesn't improve in 5 evaluations the training will automatically stop.

- Here in the evaluation we also gonna print a Sampling for us to see how is the model "talking" at that moment in order to have a more "human" vision of the improvement of the model in all the learning proccess.

8) <u>Saving Checkpoint</u>.

- Here is just a function call every 1000 iterations which will save the progress even without the need of improvement like the early stopping.

9) <u>Progress Bar</u>.

- Finally we have the progress bar which using tqdm will show us the progress and the amount of staff that its happening like the velocity of iterations which will be helpfull because we are training in CPU also the time remaining and more.

In [ ]:
for iters in progress_bar:
    
    batch_losses = []

    #__GETTING BATCHES___#

    xb, yb = get_batches('train', batch_size = 4, block_size = 8)

    
    
    #___FOWARD PASS___#
        
    x_token_and_pos = positional_encoding(xb, token_table.data, pos_table.data)

        
        
    #-------Saving Input-------#
    residual0 = x_token_and_pos     # A save that isn't a residual, for that reason I name it residual0
    #--------------------------#
        
        
    x_ln1, cache_ln1 = layer_norm(x_token_and_pos, gamma1.data, beta1.data)

    out_mha, cache_mha = multi_head_attention(x_ln1, Wq.data, Wk.data, Wv.data, Wo.data, heads = 4)
        
    x_mha = residual_connection(residual0, out_mha)
        
        
        
    #--Saving Input--#
    residual1 = x_mha 
    #----------------#
                                                  

    x_ln2, cache_ln2 = layer_norm(x_mha, gamma2.data, beta2.data)

        
        
    #-----Tensorification-------#

    x_ln2 = Tensor( x_ln2 , requires_grad = True) # "requires_grad=True" because I need it to continue backproping manually.

    residual1 = Tensor( residual1 , requires_grad = True)
        
    #---------------------------#
        
        
    Z1, A1, Z2 = FFN(x_ln2, W1, B1, W2, B2 )

    residual2 = residual_connection(residual1, Z2) 


    logits = linear_head(residual2, W_final, B_final)
        
        
    loss = cross_entropy_loss(logits, yb)
        
        
        
        
    #___ LOSS STORING____#

    batch_losses.append(loss.data)





    #____BACKPROPAGATION___#

    loss.backward()


    dout_resconn2 = backprop_residual_connection(x_ln2.grad, residual1.grad) # dout ended up as a Numpy Array.


    dout_ln2, beta2.grad, gamma2.grad = backprop_LayerNorm(dout_resconn2, cache_ln2)

        
        
    dout_MHA, Wq.grad, Wk.grad, Wv.grad, Wo.grad = backprop_MHA(dout_ln2, cache_mha)


    dout_ln1, beta1.grad, gamma1.grad = backprop_LayerNorm(dout_MHA, cache_ln1)


    dout_resconn1 = backprop_residual_connection(dout_resconn2, dout_ln1)


    pos_table.grad = backprop_positional_encoding(dout_resconn1)
        

    token_table.grad = backprop_token_embd(dout_resconn1, xb, token_table.data)




    #____ACTUALIZATION_____#


    optimizer.step()

    optimizer.zero_grad()





    avrg_loss_epoch = np.mean(batch_losses)

    loss_history["train"].append(avrg_loss_epoch)


    if iters % eval_interval == 0:

        val_loss = estimate_loss_test(batch_size = 4, block_size = 8)

        loss_history["test"].append(val_loss)



        if val_loss < best_loss :

            best_loss = val_loss

            wait = 0

            save_checkpoints(iters, optimizer, loss_history)
        
            print("checkpoint saved successfully bro")


            ##########  The cool part :) #############
            print(f"Sampling of iteration: {iters}")

            raw_generated_toketoke = generate(max_new_tokens = 30)

            print( decode(raw_generated_toketoke) ) 
            
            #########################################

        else:

            wait += 1

            if wait >= patience:
                break

    if iters % 1000 == 0:

        save_checkpoints(iters, optimizer, loss_history)




        
    progress_bar.set_postfix({ "loss": f"{avrg_loss_epoch:.4f}"} )